## Simulation of <110> dumbbells diffusion in Fe-dilute Cu alloy.
In this notebook, we use the onsager calculator we created in part 1 to compute transport coefficients for the dumbbell-mediated mechanism for dilute Cu in Fe.

The data of the energies are taken from the database given along with the paper by Messina et. al. - https://doi.org/10.1016/j.actamat.2020.03.038

We need to identify the solute-dumbbell complex states with the appropriate labels give in this paper, as well as the various dumbbell jump types. We then assign appropriate state formation and dumbbell migration energies, as per the data given in the above-mentioned database.

We then save our calculation data to compare the Green's function results for the drag and partial diffusion coefficient ratios with the results from SCMF calculations taken also from the above-mentioned database.

In part 2 and 3 for Fe - dilute Ni and dilute Si alloys, we simply use the same calculator from part 1 as here, so we don't have to re-index the states again.

In [1]:
import numpy as np

from scipy.constants import physical_constants
kB = physical_constants['Boltzmann constant in eV/K'][0]
from matplotlib import pyplot as plt

import pickle
import time

In [2]:
%%time
with open("Fe_Cu_Ni_Si.pkl","rb") as fl:
    onsagercalculator = pickle.load(fl)

CPU times: user 12min 1s, sys: 19.9 s, total: 12min 21s
Wall time: 11min 59s


### Now, we'll look at the complex states in our thermodynamic shell
We take the first state from each symmetry group of states, and print out its info in groups
of three lines.

For each state, the number printed out in the first line is the index assgined to the
symmetry group (called the "star") the state belongs to.

The second line gives the pure dumbbell orientation vector in cartesian coordinates (Recall that the length of the orientation vector is taken to be the atomic diameter of iron).

The third line prints the position of the dumbbell with respect to the solute location
in cartesian coordinates. Recall that the lattice parameter of BCC Fe is 0.2831 nm, as per Messina et al.'s paper.

Note that the first complex state (with index 0 in the first line) is an origin state (pure dumbbell on top of solute) and is unphysical.

From index 1 onwards, we'll then have to manually match our complex states with the labels in Messina et. al. (1nnA, 1nnB etc). No interaction will be assumed when a corresponding state label is not found in Messina et. al.'s database. For example, a particualr 4nn state found below was not considered in the SCMF calculations.
The first 13 such symmetry group starting from index 1 are going to contain all the states we need to consider.

In [3]:
count = 0
for star in onsagercalculator.thermo.stars[:onsagercalculator.thermo.mixedstartindex]:
    print(count)
    db = star[0].db
    print(np.round(onsagercalculator.pdbcontainer.iorlist[db.iorind][1], decimals=4)+0.)
    print(np.dot(onsagercalculator.crys.lattice, db.R))
    print()
    count += 1

0
[-0.1782  0.1782  0.    ]
[0. 0. 0.]

1
[0.     0.1782 0.1782]
[-0.14155 -0.14155 -0.14155]

2
[0.     0.1782 0.1782]
[-0.14155  0.14155 -0.14155]

3
[-0.1782  0.1782  0.    ]
[0.     0.     0.2831]

4
[-0.1782  0.1782  0.    ]
[0.     0.2831 0.    ]

5
[-0.1782  0.1782  0.    ]
[-0.2831  0.      0.2831]

6
[0.1782 0.1782 0.    ]
[-0.2831 -0.2831  0.    ]

7
[0.1782 0.1782 0.    ]
[-0.2831  0.2831  0.    ]

8
[-0.1782  0.      0.1782]
[0.14155 0.42465 0.14155]

9
[-0.1782  0.      0.1782]
[0.42465 0.14155 0.14155]

10
[0.     0.1782 0.1782]
[ 0.14155 -0.14155 -0.42465]

11
[-0.1782  0.1782  0.    ]
[-0.14155  0.14155  0.42465]

12
[0.1782 0.     0.1782]
[-0.2831 -0.2831 -0.2831]

13
[-0.1782  0.      0.1782]
[-0.2831  0.2831 -0.2831]

14
[0.1782 0.     0.1782]
[0.14155 0.42465 0.42465]

15
[0.1782 0.     0.1782]
[-0.14155  0.42465  0.42465]

16
[0.1782 0.1782 0.    ]
[-0.42465  0.42465 -0.14155]

17
[0.1782 0.1782 0.    ]
[-0.42465 -0.42465  0.14155]

18
[0.     0.1782 0.1782]
[-0.42

In [4]:
# Identify the complex states from Messina et. al. and assign corresponding labels.
name_to_themo_star = {"1nnA":2, "1nnB":1, "2nnA":3, "2nnB":4,"3nnA":7,"3nnB":5,"3nnC":6,
       "4nnA":8,"4nnB":11,"4nnC":10, "5nnA":13, "5nnB":12}

In [5]:
# sorting out the jumps with the nomenclatures for which solute-dumbbell interactions are taken into account
# in the paper by Messina et. al.
jmpdict = {"1nnA_2nnA":[], "1nnA_2nnB":[], "1nnA_3nnB":[], "1nnA_3nnC":[], "1nnB_2nnB":[], "1nnB_3nnB":[],
          "1nnB_5nnB":[], "2nnA_4nnC":[], "2nnB_4nnB":[], "2nnB_4nnC":[]}

# Now identify the jumps and put them into the dictionaries
for jlistind, jlist in enumerate(onsagercalculator.jnet1):
    
    # get the first jump of the symmetry-unique omega1 jump list
    jmp = jlist[0]
    state1 = jmp.state1
    state2 = jmp.state2
    # if rigid jump, then continue - solute-dumbbell interactions are not taken into
    # account for rigid jumps.
    if jmp.state1.db.iorind == jmp.state2.db.iorind:
        continue
    
    # get the symmetry groups ("stars") of the initial and final states
    star1 = onsagercalculator.kinetic.complexIndexdict[state1][1]
    star2 = onsagercalculator.kinetic.complexIndexdict[state2][1]
    
    # next, we see if the these stars are in the thermodynamic shell, and if the corresponding
    # jump is in our named jump dictionary above.
    if star1 in onsagercalculator.thermo2kin and star2 in onsagercalculator.thermo2kin:
        thermo_star1 = onsagercalculator.thermo.complexIndexdict[state1][1]
        thermo_star2 = onsagercalculator.thermo.complexIndexdict[state2][1]
        name1 = ""
        name2 = ""
        
        #Now see which stars the states belong to
        star1found = False
        count1 = 0
        star2found = False
        count2 = 0
        for (key, value) in name_to_themo_star.items():
            if thermo_star1==value:
                star1found = True
                count1 += 1
                name1 = key
            if thermo_star2==value:
                star2found = True
                count2 += 1
                name2 = key
        
        # just to ensure we don't have any multiple counting.
        if count1>1:
            print(thermo_star1)
        if count2>1:
            print(thermo_star2)
        
        # Now concatenate names
        jname = name1+"_"+name2
        jnameRev = name2+"_"+name1
        try:
            jmpdict[jname].append(jlistind)
        except:
            try:
                # maybe the jump we have is the reverse of what we stored as the label in the dictionary?
                jmpdict[jnamerev].append(jlistind)
            
            except:    
                continue

# check to see all necessary jumps have been found - only one symmetry group for each jump
# must be present
jmpdict

{'1nnA_2nnA': [1],
 '1nnA_2nnB': [6],
 '1nnA_3nnB': [5],
 '1nnA_3nnC': [3],
 '1nnB_2nnB': [4],
 '1nnB_3nnB': [7],
 '1nnB_5nnB': [2],
 '2nnA_4nnC': [8],
 '2nnB_4nnB': [9],
 '2nnB_4nnC': [10]}

In [6]:
# Next assign the solute-dumbbell pair states we found earlier their corresponding supercell
# energies from the database of Messina et. al.
name_to_en = {
"1nnA":-2076.29355226,
"1nnB":-2076.17073296,
"2nnA":-2076.20528052,
"2nnB":-2076.17035469,
"3nnA":-2076.12268221,
"3nnB":-2076.16878248,
"3nnC":-2076.20430949,
"4nnA":-2076.11727877,
"4nnB":-2076.17973527,
"4nnC":-2076.15460157,
"5nnA":-2076.15807100,
"5nnB":-2076.12315570
}

In [7]:
# we then compute the formation energies of the those states

E_sup_pdb = -2081.44451396
E_sup_solute = -2071.87878154 
E_bulk = -2077.21734574
E_f_pdb = 4.081701163
name_to_Ef = {}
for (key, E_IB) in name_to_en.items():
    # get the binding energy first
    Eb = -E_IB + E_sup_pdb + E_sup_solute - E_bulk
    # Next, get the formation energy (relative to solute formation energy)
    name_to_Ef[key] = E_f_pdb - Eb

# We then print out the binding energies to compare to Messina et. al.
for state, en in name_to_Ef.items():
    print("{}: {}".format(state, E_f_pdb - en))

1nnA: 0.18760249999968437
1nnB: 0.06478319999951054
2nnA: 0.09933075999970242
2nnB: 0.06440492999945491
3nnA: 0.016732449999835808
3nnB: 0.06283271999973294
3nnC: 0.09835972999962905
4nnA: 0.011329009999826667
4nnB: 0.07378550999965228
4nnC: 0.04865180999968288
5nnA: 0.05212123999945106
5nnB: 0.017205939999712427


In [8]:
# The complex energies are set. Now, we set the mixed dumbbell energies
E_b_mdb = 2075.72579114 + E_sup_pdb + E_sup_solute - E_bulk
E_f_mdb = E_f_pdb - E_b_mdb

# print formation and binding energy
E_f_mdb, E_f_mdb-E_f_pdb

(4.461859783000202, 0.38015862000020206)

In [9]:
# Next, we set the transition state energies for the omega1 jumps as per the database

# J_3_1nnA_2nnA   -2075.9362
# J_3_1nnA_2nnB   -2075.9725
# J_3_1nnA_3nnB   -2075.9056
# J_3_1nnA_3nnC   -2075.9349
# J_3_1nnB_2nnB   -2075.8653
# J_3_1nnB_3nnB   -2075.8672
# J_3_1nnB_5nnB   -2075.8141
# J_3_2nnA_4nnC   -2075.8528
# J_3_2nnB_4nnB   -2075.9148
# J_3_2nnB_4nnC   -2075.8685

Jname_2_TS_en = {"1nnA_2nnA": -2075.9362, "1nnA_2nnB": -2075.9725, "1nnA_3nnB": -2075.9056,
                 "1nnA_3nnC": -2075.9349, "1nnB_2nnB": -2075.8653, "1nnB_3nnB": -2075.8672,
                 "1nnB_5nnB": -2075.8141, "2nnA_4nnC": -2075.8528, "2nnB_4nnB": -2075.9148, 
                 "2nnB_4nnC": -2075.8685}

In [10]:
# Now, we have to find the TS energies.
Jname_2_ef_ts = {}
for (key, E_IB) in Jname_2_TS_en.items():
    Eb = -E_IB + E_sup_pdb + E_sup_solute - E_bulk
    # Next, get the formation energy (relative to solute formation energy)
    Jname_2_ef_ts[key] = E_f_pdb - Eb

In [11]:
# Then, we compute the migration energies, and print to compare with Messina et. al.
Jname_2_mig = {}
for (key, TS_en) in Jname_2_ef_ts.items():
    initstar = key[:4]
    Jname_2_mig[key] = TS_en - name_to_Ef[initstar]
Jname_2_mig

{'1nnA_2nnA': 0.35735225999997056,
 '1nnA_2nnB': 0.32105226000021503,
 '1nnA_3nnB': 0.38795226000002003,
 '1nnA_3nnC': 0.3586522599998716,
 '1nnB_2nnB': 0.3054329599999619,
 '1nnB_3nnB': 0.3035329599997567,
 '1nnB_5nnB': 0.35663295999984257,
 '2nnA_4nnC': 0.3524805199999719,
 '2nnB_4nnB': 0.2555546899998262,
 '2nnB_4nnC': 0.30185468999980003}

In [12]:
# omega2 and omega43 jumps
E_IB_43, E_IB_2 = -2075.72579114, -2075.3619
Eb_43, Eb_2 = -E_IB_43 + E_sup_pdb + E_sup_solute - E_bulk, -E_IB_2 + E_sup_pdb + E_sup_solute - E_bulk 
# Next, get the formation energy (relative to solute formation energy)
ef_ts_43 = E_f_pdb - Eb_43
ef_ts_2 = E_f_pdb - Eb_2

# print omega2, omega4, and omega3 migration energies to compare to Messina et. al.'s paper
print(ef_ts_2-E_f_mdb, ef_ts_43 - name_to_Ef["1nnB"], ef_ts_43 - E_f_mdb)

0.3638911400003053 0.4449418199997126 0.0


In [13]:
# omega2 rigid translation
E_IB_2_rigid = -2075.3573
Eb_2_rigid = -E_IB_2_rigid + E_sup_pdb + E_sup_solute - E_bulk
ef_ts_2_rigid = E_f_pdb - Eb_2_rigid
print(ef_ts_2_rigid-E_f_mdb)
print(len(onsagercalculator.jnet2))

0.3684911400000601
2


## Cu calculations

In [14]:
vu0 = 4.4447 # attempt frequencies of pure dumbbell jumps.
vu2 = 2.6848 # attempt frequencies of mixed dumbbell jumps.
Dconv=1e-2 # to change units to cm^2/s units from nm^2*THz

# pre-factors and formation energies for pure dumbbell jumps
predb0, enedb0 = np.ones(1)*np.exp(0.05), np.array([E_f_pdb])

# Here on, pre-factors are going to 1.0
# We'll measure every formation energy relative to the solute formation energy.
# so we set solute formation energy to zero, and (as per data) pre-factor to 1.
preS, eneS = np.ones(1), np.array([0.0])

# Next, interaction or the excess energies and pre-factors for solutes and dumbbells.
preSdb, eneSdb = np.ones(onsagercalculator.thermo.mixedstartindex), \
                 np.zeros(onsagercalculator.thermo.mixedstartindex)
# Now, we go over the necessary stars and assign interaction energies
for (key, index) in name_to_themo_star.items():
    eneSdb[index] = name_to_Ef[key] - E_f_pdb

predb2, enedb2 = np.ones(1), np.array([E_f_mdb])

# Transition state energies - For omega0, and omega2, the first type is the roto-translation jump,
# and the second one is the rigid jump. For omega0, the third is the on-site rotation.

# Omega0 TS eneriges
preT0, eneT0 = Dconv*vu0*np.ones(1), np.array([E_f_pdb+0.335115123, E_f_pdb + 0.61091396, E_f_pdb+0.784315123])

# Omega2 TS energies
Nj2 = len(onsagercalculator.jnet2)
preT2, eneT2 = Dconv*vu2*np.ones(Nj2), np.array([ef_ts_2, ef_ts_2_rigid])

# Omega43 TS energies
preT43, eneT43 = Dconv*vu0*np.ones(1), np.array([ef_ts_43])

# Omega1 TS energies
preT1 = Dconv*vu0*np.ones(len(onsagercalculator.jnet1))
eneT1 = np.array([eneT0[i] for i in onsagercalculator.om1types])
# Now, we go over the jumps that are provided and make the necessary changes
for (key, index) in jmpdict.items():
    eneT1[index] = Jname_2_ef_ts[key]
eneT1[0] = 0.0

In [15]:
# Now, we can begin the transport coefficient calculations
from tqdm import tqdm

temp = np.arange(200, 4001, 10)

diff_aa_Cu = np.zeros(len(temp))
diff_ab_Cu = np.zeros(len(temp))
diff_bb_Cu = np.zeros(len(temp))
diff_bb_non_loc = np.zeros(len(temp))

for i in tqdm(range(len(temp)), position=0, leave=True, ncols=65):
    T = temp[i]
    kT = kB*T
    bFdb0, bFdb2, bFS, bFSdb, bFT0, bFT1, bFT2, bFT3, bFT4 = \
        onsagercalculator.preene2betafree(kT, predb0, enedb0, preS, eneS, preSdb, eneSdb, predb2, enedb2,
                                               preT0, eneT0, preT2, eneT2, preT1, eneT1, preT43, eneT43)

    # get the transport coefficients
    # the uncorrelated (L_uc) and correlated (L_c) parts will be summed to get the total
    # transport coefficients.
    # "a" stands for solute, "b" for solvent (Fe)
    L0bb, (L_uc_aa,L_c_aa), (L_uc_bb,L_c_bb), (L_uc_ab,L_c_ab)=\
    onsagercalculator.L_ij(bFdb0, bFT0, bFdb2, bFT2, bFS, bFSdb, bFT1, bFT3, bFT4)
    
    L_aa = L_uc_aa + L_c_aa
    L_bb = L_uc_bb + L_c_bb
    L_ab = L_uc_ab + L_c_ab
    
    diff_aa_Cu[i] = L_aa[0][0]
    diff_ab_Cu[i] = L_ab[0][0]
    diff_bb_Cu[i] = L_bb[0][0]
    diff_bb_non_loc[i] = L0bb[0][0]

100%|██████████████████████████| 381/381 [42:04<00:00,  6.62s/it]


In [16]:
# Now let's do the infinite temeperature limit
kT = np.inf
bFdb0, bFdb2, bFS, bFSdb, bFT0, bFT1, bFT2, bFT3, bFT4 = \
    onsagercalculator.preene2betafree(kT, predb0, enedb0, preS, eneS, preSdb, eneSdb, predb2, enedb2,
                                           preT0, eneT0, preT2, eneT2, preT1, eneT1, preT43, eneT43)
#     bFdicts[i] = [bFdb0, bFdb2, bFS, bFSdb, bFT0, bFT1, bFT2, bFT3, bFT4]
# get the probabilities and other data from L_ij
L0bb, (L_uc_aa,L_c_aa), (L_uc_bb,L_c_bb), (L_uc_ab,L_c_ab)=\
onsagercalculator.L_ij(bFdb0, bFT0, bFdb2, bFT2, bFS, bFSdb, bFT1, bFT3, bFT4)

L_aa = L_uc_aa + L_c_aa
L_bb = L_uc_bb + L_c_bb
L_ab = L_uc_ab + L_c_ab
drag_inf = L_ab[0][0]/L_aa[0][0]
drag_inf

2.2554244079240533

In [17]:
import h5py
with h5py.File("Cu_data.h5","w") as fl:
    fl.create_dataset("diff_aa", data=diff_aa_Cu)
    fl.create_dataset("diff_ab", data=diff_ab_Cu)
    fl.create_dataset("diff_bb_nl", data=diff_bb_non_loc)
    fl.create_dataset("diff_bb", data=diff_bb_Cu)
    fl.create_dataset("drag_inf", data=np.array([drag_inf]))
    fl.create_dataset("Temp", data=temp)